In [1]:
import keras
from keras.preprocessing.image import ImageDataGenerator 

In [2]:
#Define the parameters/arguments for ImageDataGenerator class
train_datagen=ImageDataGenerator(rescale=1./255,shear_range=0.2,rotation_range=180,zoom_range=0.2,horizontal_flip=True)

test_datagen=ImageDataGenerator(rescale=1./255)

In [4]:
#Applying ImageDataGenerator functionality to trainset
x_train=train_datagen.flow_from_directory('/content/Dataset/Dataset/train_set',target_size=(128,128),batch_size=32,class_mode='binary')

Found 436 images belonging to 2 classes.


In [5]:
#Applying ImageDataGenerator functionality to testset
x_test=test_datagen.flow_from_directory('/content/Dataset/Dataset/test_set',target_size=(128,128),batch_size=32,class_mode='binary')

Found 121 images belonging to 2 classes.


In [6]:
#import model building libraries

#To define Linear initialisation import Sequential
from keras.models import Sequential
#To add layers import Dense
from keras.layers import Dense
#To create Convolution kernel import Convolution2D
from keras.layers import Convolution2D
#import Maxpooling layer
from keras.layers import MaxPooling2D
#import flatten layer
from keras.layers import Flatten
import warnings
warnings.filterwarnings('ignore')

In [7]:
#initializing the model
model=Sequential()

In [8]:
#add convolutional layer
model.add(Convolution2D(32,(3,3),input_shape=(128,128,3),activation='relu'))
#add maxpooling layer
model.add(MaxPooling2D(pool_size=(2,2)))
#add flatten layer 
model.add(Flatten()) 

In [9]:
#add hidden layer
model.add(Dense(150,activation='relu'))
#add output layer
model.add(Dense(1,activation='sigmoid'))

In [10]:
#configure the learning process
model.compile(loss='binary_crossentropy',optimizer="adam",metrics=["accuracy"])

In [12]:
#Training the model
model.fit_generator(x_train,steps_per_epoch=14,epochs=10,validation_data=x_test,validation_steps=4)

Epoch 1/10
14/14 [==============================] - 26s 2s/step - loss: 0.1884 - accuracy: 0.9128 - val_loss: 0.0690 - val_accuracy: 0.9669
Epoch 2/10
14/14 [==============================] - 25s 2s/step - loss: 0.2131 - accuracy: 0.8830 - val_loss: 0.0923 - val_accuracy: 0.9504
Epoch 3/10
14/14 [==============================] - 25s 2s/step - loss: 0.1947 - accuracy: 0.9151 - val_loss: 0.0740 - val_accuracy: 0.9587
Epoch 4/10
14/14 [==============================] - 25s 2s/step - loss: 0.1663 - accuracy: 0.9312 - val_loss: 0.0698 - val_accuracy: 0.9752
Epoch 5/10
14/14 [==============================] - 26s 2s/step - loss: 0.1668 - accuracy: 0.9404 - val_loss: 0.0611 - val_accuracy: 0.9835
Epoch 6/10
14/14 [==============================] - 25s 2s/step - loss: 0.1840 - accuracy: 0.9151 - val_loss: 0.0641 - val_accuracy: 0.9752
Epoch 7/10
14/14 [==============================] - 25s 2s/step - loss: 0.2018 - accuracy: 0.9128 - val_loss: 0.0846 - val_accuracy: 0.9752
Epoch 8/10
14/14 [==

In [13]:
model.save("forest1.h5")

In [14]:
#import load_model from keras.model
from keras.models import load_model
#import image class from keras
from tensorflow.keras.preprocessing import image
#import numpy
import numpy as np
#import cv2
import cv2


In [15]:
#load the saved model
model = load_model("forest1.h5")

In [16]:
img=image.load_img('/content/Dataset/Dataset/test_set/with fire/180802_CarrFire_010_large_700x467.jpg')
x=image.img_to_array(img)
res = cv2.resize(x, dsize=(128, 128), interpolation=cv2.INTER_CUBIC)
#expand the image shape
x=np.expand_dims(res,axis=0)

In [17]:
pred=model.predict(x)

1/1 [==============================] - 0s 142ms/step


In [18]:
pred

array([[1.]], dtype=float32)

In [23]:
pip install twilio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.4 MB 5.3 MB/s 


In [28]:
pip install pygobject

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [35]:
#import opencv library
import cv2
#import numpy
import numpy as np
#import image function from keras
from tensorflow.keras.preprocessing import image
#import load_model from keras
from keras.models import load_model
#import Client from twilio API
from twilio.rest import Client
#import playsound package
from playsound import playsound

In [ ]:
#load the saved model
model=load_model('forest1.h5')
#define video
video=cv2.VideoCapture(0)
#define the features
name=['forest','with fire']
while(1):
  success,frame=video.read()
  cv2.imwrite("image.jpg",frame)
  img=image.load_img("image.jpg",target_size=(64,64))
  x=image.img_to_array(img)
  res = cv2.resize(x, dsize=(128, 128), interpolation=cv2.INTER_CUBIC)
  x=np.expand_dims(res,axis=0)
  pred=model.predict(x)
  p=pred[0]
  print(pred)
  #cv2.putText(frame,"predicted class = "+str(name[p]),(100,100),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,0))
  if pred[0]==1:
    #twilio account ssid
    account_sid='ACb252d719e2b09dbb3f44dd2f8c8be56a'
    #twilio account authentication token
    auth_token ='d401dbcf96a018136bc7ad3ded613273'
    client=Client(account_sid,auth_token)

    message=client.messages \
    .create(
        body='Forest Fire is detected,stay alert',
        #use twilio free number
        from_='+1 980 414 5862',
        #to number
        to='+91 9080590163')
    print(message.sid)
    print('Fire Detected')
    print('SMS sent!')
    playsound('/tornado-siren-in-streamwood-il-35510.mp3')
  else:
    print('No Danger')
    #break
  cv2.imshow("image",frame)
  if cv2.waitkey(1) & 0xFF == ord('a'):
     break
video.release()
cv2.destroyAllWindows()
